In [4]:
# Import dependencies
# !pip install yfinance
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import yfinance as yf

In [11]:
# Load data
data = yf.download("^GSPC", start = '2001-01-03', end='2005-12-31')

[*********************100%***********************]  1 of 1 completed


In [30]:
#Get only adj Close column and get the change
df = data['Adj Close'].pct_change()*100
df = df.rename("Today")
df = df.reset_index()
df

,Date,Today
0,2001-01-03,NaN
1,2001-01-04,-1.055247
2,2001-01-05,-2.624236
3,2001-01-08,-0.191781
4,2001-01-09,0.381219
...,...,...
1250,2005-12-23,0.042586
1251,2005-12-27,-0.955338
1252,2005-12-28,0.129722
1253,2005-12-29,-0.298052


In [31]:
# Create a lag 5 times
for i in range(1,6):
    df['Lag '+str(i)] = df['Today'].shift(i)
df

,Date,Today,Lag 1,Lag 2,Lag 3,Lag 4,Lag 5
0,2001-01-03,NaN,NaN,NaN,NaN,NaN,NaN
1,2001-01-04,-1.055247,NaN,NaN,NaN,NaN,NaN
2,2001-01-05,-2.624236,-1.055247,NaN,NaN,NaN,NaN
3,2001-01-08,-0.191781,-2.624236,-1.055247,NaN,NaN,NaN
4,2001-01-09,0.381219,-0.191781,-2.624236,-1.055247,NaN,NaN
...,...,...,...,...,...,...,...
1250,2005-12-23,0.042586,0.422078,0.251667,-0.023815,-0.583902,-0.284828
1251,2005-12-27,-0.955338,0.042586,0.422078,0.251667,-0.023815,-0.583902
1252,2005-12-28,0.129722,-0.955338,0.042586,0.422078,0.251667,-0.023815
1253,2005-12-29,-0.298052,0.129722,-0.955338,0.042586,0.422078,0.251667


In [32]:
df['Volume'] = data.Volume.shift(1).values/1000_000_000
df = df.dropna()
df

,Date,Today,Lag 1,Lag 2,Lag 3,Lag 4,Lag 5,Volume
6,2001-01-11,1.031770,0.958639,0.381219,-0.191781,-2.624236,-1.055247,1.29650
7,2001-01-12,-0.623287,1.031770,0.958639,0.381219,-0.191781,-2.624236,1.41120
8,2001-01-16,0.614309,-0.623287,1.031770,0.958639,0.381219,-0.191781,1.27600
9,2001-01-17,0.212561,0.614309,-0.623287,1.031770,0.958639,0.381219,1.20570
10,2001-01-18,1.391532,0.212561,0.614309,-0.623287,1.031770,0.958639,1.34910
...,...,...,...,...,...,...,...,...
1250,2005-12-23,0.042586,0.422078,0.251667,-0.023815,-0.583902,-0.284828,1.88850
1251,2005-12-27,-0.955338,0.042586,0.422078,0.251667,-0.023815,-0.583902,1.28581
1252,2005-12-28,0.129722,-0.955338,0.042586,0.422078,0.251667,-0.023815,1.54047
1253,2005-12-29,-0.298052,0.129722,-0.955338,0.042586,0.422078,0.251667,1.42236


In [33]:
# Direction column, when the change is + UP (1), - DOWN (0)
df['Direction'] = ["UP" if i>0 else "DOWN" for i in df['Today']]

<ipython-input-33-cebc6c13b524>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Direction'] = ["UP" if i>0 else "DOWN" for i in df['Today']]


In [34]:
df

,Date,Today,Lag 1,Lag 2,Lag 3,Lag 4,Lag 5,Volume,Direction
6,2001-01-11,1.031770,0.958639,0.381219,-0.191781,-2.624236,-1.055247,1.29650,UP
7,2001-01-12,-0.623287,1.031770,0.958639,0.381219,-0.191781,-2.624236,1.41120,DOWN
8,2001-01-16,0.614309,-0.623287,1.031770,0.958639,0.381219,-0.191781,1.27600,UP
9,2001-01-17,0.212561,0.614309,-0.623287,1.031770,0.958639,0.381219,1.20570,UP
10,2001-01-18,1.391532,0.212561,0.614309,-0.623287,1.031770,0.958639,1.34910,UP
...,...,...,...,...,...,...,...,...,...
1250,2005-12-23,0.042586,0.422078,0.251667,-0.023815,-0.583902,-0.284828,1.88850,UP
1251,2005-12-27,-0.955338,0.042586,0.422078,0.251667,-0.023815,-0.583902,1.28581,DOWN
1252,2005-12-28,0.129722,-0.955338,0.042586,0.422078,0.251667,-0.023815,1.54047,UP
1253,2005-12-29,-0.298052,0.129722,-0.955338,0.042586,0.422078,0.251667,1.42236,DOWN


In [ ]:
## Min 10:06

In [35]:
# https://www.youtube.com/watch?v=X9jjyh0p7x8&list=PLq28f6fmT3iLVFdBnzEwQS1GZ5-QLvU2W&index=2&t=1373s&ab_channel=Algovibes

In [23]:
# Transform the data set into a data frame
# data = the date we want/independent variables/X value
# feature_names = the column names of the data
# target = the target variable/price of houses/dependent variable/ Y value

df_x = pd.DataFrame(boston.data, columns = boston.feature_names)
df_y = pd.DataFrame(boston.target)

In [24]:
# Initialise the linear regresseion model
reg = linear_model.LinearRegression()

In [26]:
# Split the data into 67% training and 33% testing data
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size = 0.33, random_state = 42)

In [27]:
# Train Model with our trainin data
reg.fit(x_train, y_train)

LinearRegression()

In [35]:
# Print the predictions on our test data
y_pred = reg.predict(x_test)
print(y_pred)

# Actual values
# print(y_test)

In [ ]:
# Check the model performance / accuracy is